This notebook is used to make plots. The notebook is converted to a plain python text file using [jupytext](https://jupytext.readthedocs.io/en/latest/)

In [4]:
import numpy as np
import pandas as pd

In [13]:
aa_a_1k = pd.read_csv("csv_files/Active learning results - new - AA_A_1k.csv")


In [15]:
aa_a_1k.mean_vals

0     0.547348
1     0.405195
2     0.345960
3     0.304385
4     0.277288
5     0.257633
6     0.241584
7     0.226030
8     0.216310
9     0.209477
10    0.201103
11    0.193271
12    0.186277
13    0.178212
14    0.172891
15    0.167611
16    0.164436
Name: mean_vals, dtype: float64

In [2]:
!python -m jupytext --sync MakePlots.ipynb

[jupytext] Reading MakePlots.ipynb in format ipynb
[jupytext] Loading MakePlots.py
[jupytext] Updating the timestamp of MakePlots.py
